In [1]:
import os
import socket
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import udf, length, when, col, lit, broadcast
from pyspark.sql.types import BooleanType, IntegerType, LongType, StringType, ArrayType, FloatType, StructType, StructField
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
from pyspark import StorageLevel
from jinja2 import Environment, FileSystemLoader

from pyspark.sql.functions import * # count, desc, size, explode, arrays_zip, regexp_extract

In [2]:
# import sys
# !{sys.executable} -m pip install --user --trusted-host pypi-registry.supplementary-services.svc.cluster.local --index http://pypi-registry.supplementary-services.svc.cluster.local:8080/ emoji

In [3]:
from emoji import get_emoji_regexp, unicode_codes
import re

In [4]:
# setting constants
APP_NAME = "YOUR_APP_NAME"
NORMALIZED_APP_NAME = APP_NAME.replace('/', '_').replace(':', '_')

APPS_TMP_DIR = os.path.join(os.getcwd(), "tmp")
APPS_CONF_DIR = os.path.join(os.getcwd(), "conf")
APPS_LOGS_DIR = os.path.join(os.getcwd(), "logs")
LOG4J_PROP_FILE = os.path.join(APPS_CONF_DIR, "pyspark-log4j-{}.properties".format(NORMALIZED_APP_NAME))
LOG_FILE = os.path.join(APPS_LOGS_DIR, 'pyspark-{}.log'.format(NORMALIZED_APP_NAME))
EXTRA_JAVA_OPTIONS = "-Dlog4j.configuration=file://{} -Dspark.hadoop.dfs.replication=1 -Dhttps.protocols=TLSv1.0,TLSv1.1,TLSv1.2,TLSv1.3"\
    .format(LOG4J_PROP_FILE)

LOCAL_IP = socket.gethostbyname(socket.gethostname())

In [5]:
# preparing configuration files from templates
for directory in [APPS_CONF_DIR, APPS_LOGS_DIR, APPS_TMP_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)

env = Environment(loader=FileSystemLoader('/opt'))
template = env.get_template("pyspark_log4j.properties.template")
template\
    .stream(logfile=LOG_FILE)\
    .dump(LOG4J_PROP_FILE)

In [6]:
# run spark
spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master("k8s://https://10.32.7.103:6443")\
    .config("spark.driver.host", LOCAL_IP)\
    .config("spark.ui.port", "4040")\
    .config("spark.kubernetes.memoryOverheadFactor", "0.6")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.bindAddress", "0.0.0.0")\
    .config("spark.executor.instances", "5")\
    .config("spark.executor.cores", '4')\
    .config("spark.executor.memory", "5g")\
    .config("spark.memory.fraction", "0.6")\
    .config("spark.memory.storageFraction", "0.5")\
    .config("spark.sql.autoBroadcastJoinThreshold", "-1")\
    .config("spark.driver.extraJavaOptions", EXTRA_JAVA_OPTIONS)\
    .config("spark.kubernetes.namespace", "sburmistrova-266294")\
    .config("spark.kubernetes.driver.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.label.appname", APP_NAME)\
    .config("spark.kubernetes.container.image.pullPolicy", "Always")\
    .config("spark.kubernetes.container.image", "node03.st:5000/spark-executor:sburmistrova-266294")\
    .config("spark.kubernetes.executor.deleteOnTermination", "false")\
    .config("spark.local.dir", "/tmp/spark")\
    .getOrCreate()

In [7]:
spark.version

'3.1.1'

## Task 1

Find the top 20 posts in the group: 
* by likes; 
* by comments; 
* by reposts.

In [8]:
posts_df = spark.read.json("/shared/bigdata20/posts_api.json")

In [9]:
posts_likes_df = spark.read.parquet("/shared/bigdata20/posts_likes.parquet")
followers_df = spark.read.parquet("/shared/bigdata20/followers.parquet")
followers_posts_df = spark.read.json("/shared/bigdata20/followers_posts_api_final.json")
followers_posts_likes_df = spark.read.parquet("/shared/bigdata20/followers_posts_likes.parquet")

In [10]:
posts_df.printSchema()

root
 |-- attachments: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- album: struct (nullable = true)
 |    |    |    |-- created: long (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |-- size: long (nullable = true)
 |    |    |    |-- thumb: struct (nullable = true)
 |    |    |    |    |-- access_key: string (nullable = true)
 |    |    |    |    |-- album_id: long (nullable = true)
 |    |    |    |    |-- date: long (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |    |-- sizes: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |    |-- type: string (nullable = true)
 |   

In [11]:
# by likes
posts_df.select("id", col("likes.count").alias("count_likes"))\
                .orderBy("count_likes", ascending=False).show()

+-----+-----------+
|   id|count_likes|
+-----+-----------+
|32022|       1637|
|35068|       1629|
|17492|       1516|
|18526|       1026|
|19552|        955|
|41468|        952|
|19419|        868|
|29046|        824|
|32546|        786|
|24085|        765|
|40180|        759|
|33658|        708|
|13532|        633|
|40842|        631|
|35117|        588|
|17014|        581|
|19583|        553|
|19809|        552|
|27455|        550|
|11999|        549|
+-----+-----------+
only showing top 20 rows



In [12]:
# by comments
posts_df.select("id", col('comments.count').alias('comments_count'))\
.orderBy("comments_count", ascending=False).show()

+-----+--------------+
|   id|comments_count|
+-----+--------------+
|24085|           850|
|22540|           250|
|27722|           192|
| 8285|           148|
|26860|           113|
|13571|           107|
|39294|           104|
|36680|            96|
|26006|            92|
|41739|            92|
|12426|            91|
|21499|            88|
|39163|            83|
|39407|            83|
|11267|            81|
|31548|            80|
|11158|            70|
|39082|            67|
|14602|            61|
|12687|            61|
+-----+--------------+
only showing top 20 rows



In [13]:
# by reposts
posts_df.select("id", col("reposts.count").alias("reposts_count"))\
.orderBy("reposts_count", ascending=False).show()

+-----+-------------+
|   id|reposts_count|
+-----+-------------+
|17492|          334|
|19552|          246|
|32022|          210|
|11842|          129|
|19419|          126|
|13532|          110|
|17014|          105|
|35068|          101|
|41266|           92|
|12593|           90|
|29046|           87|
|11999|           85|
|41468|           85|
|19809|           84|
|17167|           81|
|10833|           78|
|18543|           77|
|16596|           76|
|18156|           74|
|37262|           71|
+-----+-------------+
only showing top 20 rows



## Task 2

Find the top 20 users by 
* likes
* reposts they have made 

(to trace reposts use "*copy_history*" field)

In [14]:
# by likes
posts_likes_df.printSchema()

root
 |-- itemType: string (nullable = true)
 |-- ownerId: integer (nullable = true)
 |-- itemId: integer (nullable = true)
 |-- likerId: integer (nullable = true)



In [15]:
posts_likes_df.groupBy("likerId")\
.agg(count("itemId").alias("likes_count"))\
.orderBy("likes_count", ascending=False).show()

+---------+-----------+
|  likerId|likes_count|
+---------+-----------+
|  2070090|       4801|
|  2397858|       2055|
|  1475301|       1829|
|    18239|       1569|
|   546612|       1245|
|     6371|        907|
|  1841959|        746|
| 78440957|        709|
|   120248|        699|
| 40981497|        611|
|    22158|        553|
|207628162|        548|
|329377723|        504|
| 76071304|        474|
| 14805173|        440|
|   317799|        385|
| 56355640|        375|
| 52042971|        338|
|  7437271|        336|
|136506644|        335|
+---------+-----------+
only showing top 20 rows



In [16]:
# by reposts
followers_posts_df.select("owner_id", col("copy_history.id").getItem(0).alias("post_id"))\
.na.drop().groupby("owner_id")\
.agg(count("post_id").alias("posts_num"))\
.orderBy("posts_num", ascending=False).show()

+---------+---------+
| owner_id|posts_num|
+---------+---------+
|  2547211|    37742|
|357231922|    23349|
|168543860|    18429|
| 25646344|    11122|
|176861294|     9022|
|524656784|     7242|
|    29840|     7164|
|143207077|     7161|
|141687240|     6804|
|459339006|     6741|
|514384760|     6570|
|483715951|     6052|
|445159771|     5808|
|451211328|     5646|
|426396104|     5533|
|  8325325|     5532|
|452280411|     5458|
|464220898|     5318|
|440454268|     5304|
|461319529|     5240|
+---------+---------+
only showing top 20 rows



## Task 3

Get reposts of the original posts of the itmo group (*posts.json*) from user posts 
(the result should be similar to (*group_post_id*, *Array(user_post_ids)*))

In [17]:
followers_posts_df.select(col("id").name("user_post_id"),\
                        col("copy_history.id").getItem(0).name("post_id"),\
                        col("copy_history.owner_id").getItem(0).name("owner_id"))\
                  .join(posts_df.select(col("id").name("post_id"), "owner_id"),\
                        ["owner_id", "post_id"])\
                  .groupBy("post_id")\
                  .agg(collect_list('user_post_id').name('user_post_ids'))\
                  .withColumn('reposts_num', size('user_post_ids'))\
                  .orderBy('reposts_num', ascending=False)\
                  .show()


+-------+--------------------+-----------+
|post_id|       user_post_ids|reposts_num|
+-------+--------------------+-----------+
|  41266|[1265, 1748, 88, ...|         30|
|  41468|[89, 1427, 629, 3...|         25|
|  42482|[1760, 12003, 420...|         10|
|  40090|[349, 1760, 463, ...|          9|
|  39259|[14840, 3560, 175...|          8|
|  38740|[2649, 9496, 3190...|          8|
|  41207|[4812, 2960, 958,...|          6|
|  41721|[274, 4865, 8, 28...|          6|
|  41546|[1161, 3908, 7005...|          6|
|  38963|[3720, 6403, 4868...|          5|
|  41506|[670, 12823, 135,...|          5|
|  39682|[3262, 3368, 384,...|          5|
|  41708|[10107, 708, 271,...|          4|
|  39515|[2321, 7390, 443,...|          4|
|  41108|[10483, 366, 1055...|          4|
|  38915|[1186, 4704, 4487...|          4|
|  39294|[4718, 939, 2319,...|          4|
|  42730|[5561, 6840, 4201...|          4|
|  39686|[3370, 4741, 182,...|          4|
|  40011| [4770, 19267, 2559]|          3|
+-------+--

## Task 4

find emoticons in posts (negative, positive, neutral),
* calculate their overall count,
* frequency (number of posts they can be found in)
* average count per post. 

 - Print top 10 most popular emoticons, 
 - print top 5 emoticons which have the greatest difference between  their overall count and frequency,
 - print top 5 emoticons with average count per post.

In [18]:
@udf(returnType=ArrayType(StringType()))
def get_emoji_udf(text):
    all_matches = get_emoji_regexp().finditer(text)
    emoji = []
    for match in all_matches:
        emoji.append(match.group())
    
    return emoji

In [19]:
#find emoticons in posts, and their statistics
emoji_df = posts_df.where("text <> ''")\
.select('id', get_emoji_udf(col("text")).name('emoji'))\
.where(size('emoji') > 0)\
.withColumn('emoji', explode("emoji")).groupBy('emoji')\
.agg(count('id').name('all_count'), countDistinct('id')\
.name('posts_count'))\
.withColumn('avg_per_post', col('all_count') / col('posts_count'))\
.withColumn('difference', col('all_count') - col('posts_count'))\
.orderBy(desc('all_count'))

emoji_df.limit(5).toPandas()


,emoji,all_count,posts_count,avg_per_post,difference
0,🔥,76,61,1.245902,15
1,⚡,68,45,1.511111,23
2,📍,63,39,1.615385,24
3,🚀,50,47,1.063830,3
4,❤,47,47,1.000000,0


In [20]:
#Print top 10 most popular emoticons
emoji_df.limit(10).toPandas()

,emoji,all_count,posts_count,avg_per_post,difference
0,🔥,76,61,1.245902,15
1,⚡,68,45,1.511111,23
2,📍,63,39,1.615385,24
3,🚀,50,47,1.063830,3
4,❤,47,47,1.000000,0
5,❗,45,20,2.250000,25
6,🔵,40,17,2.352941,23
7,⬇,40,16,2.500000,24
8,✔,40,9,4.444444,31
9,💙,38,38,1.000000,0


In [21]:
#print top 5 emoticons which have the greatest difference 
#between their overall count and frequency
emoji_df.orderBy(desc('difference')).limit(5).toPandas()

,emoji,all_count,posts_count,avg_per_post,difference
0,✔,40,9,4.444444,31
1,❗,45,20,2.250000,25
2,📍,63,39,1.615385,24
3,⬇,40,16,2.500000,24
4,🔵,40,17,2.352941,23


In [22]:
#print top 5 emoticons with average count per post
emoji_df.orderBy(desc('avg_per_post')).limit(5).toPandas()

,emoji,all_count,posts_count,avg_per_post,difference
0,✔,40,9,4.444444,31
1,🔹,30,7,4.285714,23
2,▪,21,5,4.200000,16
3,✒,7,2,3.500000,5
4,📷,3,1,3.000000,2


## Task 5

Probable "fans". Find for each user the top 10 other users whose posts this user likes.

In [23]:
followers_posts_likes_df.select(col('ownerId').alias('user_id'), 'itemId', 'likerId')\
.groupBy('likerId', 'user_id')\
.agg(count('itemId').alias("likes_count"))\
.withColumn("top", row_number().over(Window.partitionBy("likerId").orderBy(desc("likes_count"))))\
.where('top <= 10')\
.orderBy('likerId', 'top')\
.show()

+-------+--------+-----------+---+
|likerId| user_id|likes_count|top|
+-------+--------+-----------+---+
|      9|  654356|          4|  1|
|     14| 1986125|          2|  1|
|     14|   35524|          1|  2|
|     14| 3680017|          1|  3|
|     14|    3420|          1|  4|
|     15|  473831|          2|  1|
|     15| 1227412|          2|  2|
|     17| 1986125|         10|  1|
|     17|    3420|          3|  2|
|     17|   88060|          2|  3|
|     34| 1986125|          1|  1|
|     34|   50601|          1|  2|
|     63|   73115|          1|  1|
|     99| 1550591|          1|  1|
|    122|   55983|          1|  1|
|    143|     637|          3|  1|
|    146|19515730|          3|  1|
|    146|  106610|          2|  2|
|    149| 1986125|          1|  1|
|    154| 1601367|          1|  1|
+-------+--------+-----------+---+
only showing top 20 rows



## Task 6

Probable friends. If two users like each other posts they may be friends. 

Find pairs of users where both users are top likers of each other.

In [25]:
user_df_1 = followers_posts_likes_df\
.select(col("ownerId").name("user_1"), col("likerId").alias("user_2"), col("itemId"))

user_df_2 = followers_posts_likes_df\
.select(col("ownerId").name("user_2"), col("likerId").alias("user_1"))

user_df_3 = user_df_1.join(user_df_2, ["user_1", "user_2"]).where("user_1 != user_2")\
.groupBy("user_1", "user_2").agg(count("itemId").alias("cnt_posts"))\
.orderBy(desc("cnt_posts"))

In [26]:
user_df_3.withColumn('first_user', least('user_1', 'user_2'))\
.withColumn('second_user', greatest('user_1', 'user_2'))\
.drop('user_1', 'user_2').dropDuplicates()\
.orderBy('cnt_posts', ascending=False).show()

+---------+----------+-----------+
|cnt_posts|first_user|second_user|
+---------+----------+-----------+
|    16100|  13675440|  183535934|
|     1972|   1475301|    5633955|
|     1612| 207134315|  208946862|
|     1456| 145105762|  267301242|
|      880|    108408|    7697818|
|      810|    135451|   18737802|
|      720| 209077977|  272076217|
|      684| 155963006|  162366815|
|      608|   2547211|    4448812|
|      598|  19261491|  229861638|
|      572|  53368685|  322831238|
|      544|  52612744|   53720099|
|      506|  66022003|   95356919|
|      390|  83892412|  115252127|
|      376|  47122493|   63363182|
|      330|  44770563|  103229751|
|      330|  27102997|   74634237|
|      312| 101767883|  188548515|
|      300|   1475301|    4068532|
|      299|  34892097|   59949877|
+---------+----------+-----------+
only showing top 20 rows



In [ ]:
#spark.stop()

In [27]:
!pip list

Package                       Version
----------------------------- -------------------
alembic                       1.5.8
anyio                         2.2.0
appdirs                       1.4.4
argon2-cffi                   20.1.0
async-generator               1.10
attrs                         20.3.0
Babel                         2.9.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.1
beautifulsoup4                4.9.3
bleach                        3.3.0
blinker                       1.4
bokeh                         2.3.0
Bottleneck                    1.3.2
brotlipy                      0.7.0
cached-property               1.5.2
certifi                       2020.12.5
certipy                       0.1.3
cffi                          1.14.5
chardet                       4.0.0
click                         7.1.2
cloudpickle                   1.6.0
conda                         4.9.2
conda-package-handling        1.7.2
cryptography                  3.4.6
cycler  